# Example for a parameter estimation

In [1]:
from CADETProcess.comparison import Comparator
from CADETProcess.reference import ReferenceIO
import numpy as np

time  = np.array([0,1,2,3,4])
concentrations = np.array([15, 13.60707976,11.37628152,10.48811636, 9.72366553 ])
comparator = Comparator()




reference = ReferenceIO('dextran experiment', time, concentrations)

comparator.add_difference_metric(
    'SSE', reference, 'column.outlet', start=5*60, end=7*60
)

# metrices = "shape metric"